In [ ]:
import os
from dotenv import load_dotenv

gemini_api_key = os.getenv('GEMINI_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

#### Loading LLM using init method

In [3]:
from langchain.chat_models import init_chat_model

gemini_model = init_chat_model(model_provider='google_genai',
                               model='gemini-2.0-flash-001',
                               )

gemini_model.invoke('Explain machine learning in 1 line.').content

'Machine learning enables computers to learn from data without explicit programming.'

#### Loading LLM using configurable method

In [8]:
gemini_configurable_model = init_chat_model()

gemini_configurable_model.invoke('What is the capital of gujarat?', config={"configurable": {"model": "gemini-2.0-flash-001", "model_provider": "google_genai"}}).content

'The capital of Gujarat is **Gandhinagar**.'

#### Tool Calling

In [37]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool

@tool
class add(BaseModel):
    """Add two numbers."""

    a: int = Field(..., description="First Integer")
    b: int = Field(..., description="Second Integer")

In [38]:
gemini_with_tools = gemini_model.bind_tools([add])

output = gemini_with_tools.invoke('What is 44 + 6 ?')

In [39]:
print(output)

content='' additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 44.0, "b": 6.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []} id='run--89f18f5e-fb4a-4b94-980e-ed5189d301e4-0' tool_calls=[{'name': 'add', 'args': {'a': 44.0, 'b': 6.0}, 'id': '26a29bbf-ff5f-436c-bac6-9c9596e21079', 'type': 'tool_call'}] usage_metadata={'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


In [40]:
output.tool_calls

[{'name': 'add',
  'args': {'a': 44.0, 'b': 6.0},
  'id': '26a29bbf-ff5f-436c-bac6-9c9596e21079',
  'type': 'tool_call'}]

#### Passing tool output to LLM

In [41]:
from langchain_core.messages import HumanMessage

query = "What is 3 + 97 ? and What is 99 + 1"
messages = [HumanMessage(query)]

ai_msg = gemini_with_tools.invoke(messages)
print(ai_msg.tool_calls)
messages.append(ai_msg)

[{'name': 'add', 'args': {'a': 3.0, 'b': 97.0}, 'id': 'f4e5db69-1eaf-4651-9fde-a489ab917f5d', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 99.0, 'b': 1.0}, 'id': '1a36ae25-fdf0-4dde-85b7-6bbd477a0da7', 'type': 'tool_call'}]


In [43]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 + 97 ? and What is 99 + 1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 99.0, "b": 1.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--e76802fb-a2b5-498b-b093-2fac2b2de0c9-0', tool_calls=[{'name': 'add', 'args': {'a': 3.0, 'b': 97.0}, 'id': 'f4e5db69-1eaf-4651-9fde-a489ab917f5d', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 99.0, 'b': 1.0}, 'id': '1a36ae25-fdf0-4dde-85b7-6bbd477a0da7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='a=3 b=97', name='add', tool_call_id='f4e5db69-1eaf-4651-9fde-a489ab917f5d'),
 ToolMessage(content='a=99 b=1', name='add', tool_call_id='1a36ae25-fdf0-4dde-85b7-6bbd477a0da7')]

In [44]:
gemini_with_tools.invoke(messages).content

'3 + 97 = 100 and 99 + 1 = 100'

#### Structured Output from LLM

In [52]:
from typing import Optional
from pydantic import BaseModel, Field

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(..., description="The setup of the joke")
    punchline: str = Field(..., description="The punchline of the joke")
    rating: Optional[int] = Field(default=None, description="How funny the joke is, from 1 to 10")

In [56]:
structured_llm = gemini_model.with_structured_output(Joke)
structured_llm.invoke("Tell me an joke about horse")

Joke(setup='Why did the horse quit his job?', punchline='Because he already has a stable job!', rating=None)